## Load tool data

* Connection info has to be added to the hidden `.env` file in the root folder

In [10]:
import os
import requests
from dotenv import load_dotenv
from requests.auth import HTTPBasicAuth
from bs4 import BeautifulSoup
from datetime import datetime
import pandas as pd

load_dotenv('../.env')

data = []

for year in [2023,2024]:
    for month in range(1,11):
        url = f'{os.getenv("TOOL_URL")}?platz=metzingen&year={year}&month={month}'
        #print(url)
        r = requests.get(url, auth=HTTPBasicAuth(os.getenv("TOOL_USR"), os.getenv("TOOL_PWD")))
        if r.status_code==200:
            #print(len(r.content))
            soup = BeautifulSoup(r.content, 'html.parser')
            #print(soup)
            for tbl in soup.findAll('table'):
                #print(len(tbl))
                trs = tbl.findAll('tr',attrs={'class':None})
                if len(trs) > 0:
                    first=trs[0]
                    dateTxt = first.find('th').text
                    last= trs[-1]
                    datarow =[ dateTxt[:3],datetime.strptime(dateTxt[3:],' %d.%m.%y'), last.find('td', attrs={'colspan':9}).text ]
                    data.append(datarow)

df_tool = pd.DataFrame(data, columns=['Weekday','Date','Message'])

df_tool.tail()

,Weekday,Date,Message
300,Sun,2024-10-27,war wohl kein Flugwetter
301,Mon,2024-10-28,war wohl kein Flugwetter
302,Tue,2024-10-29,"vielleicht nur 3 Std. Flugwetter, grenzwertig"
303,Wed,2024-10-30,war wohl kein Flugwetter
304,Thu,2024-10-31,war wohl kein Flugwetter


## Get DHV XC data

In [11]:
import requests
import urllib

PAGE_SIZE = 500

places = {
    'Metzingen':11185,
    'Rammelsberg': 9427,
    'Estorf': 11001,
    'Leese': 10746,
    'Porta': 9712,
    'Lüdingen':9759,
    'Brunsberg': 9844,
    'Kella': 9521,
    'Börry': 9403
}

results = []
for place in ['Metzingen']:
    limit = PAGE_SIZE
    for start in [0,PAGE_SIZE]:
        query = {"navpars":{"start":start,"limit":limit,"sort":[{"field":"FlightDate","dir":-1},{"field":"BestTaskPoints","dir":-1}]}}
        # for some reason, " is replaced by ' and spaces are added which is replaced by +
        decoded_url = f"https://de.dhv-xc.de/api/fli/flights?s?fkcat%5B%5D=1&fkto%5B%5D={places[place]}&{urllib.parse.urlencode(query,quote_via=urllib.parse.quote_plus).replace('%27', '%22').replace('+', '')}"
        #print(decoded_url)
        r = requests.get(decoded_url)
        if r.status_code==200:
            response = r.json()
            df = pd.DataFrame(response['data'])
            #print(df.columns.values)
            results.extend(df[['FlightDate', 'TakeoffWaypointName' , 'Glider' ,'FlightDuration','UtcOffset', 'FlightStartTime']].values)


df_results = pd.DataFrame(results,columns=['FlightDate','Takeoff', 'Glider' ,'Duration','UtcOffset', 'FlightStartTime'])
print(len(df_results))
df_results.tail()

1000


,FlightDate,Takeoff,Glider,Duration,UtcOffset,FlightStartTime
995,2018-05-26,Metzingen,Anakis3 - XL [LTF 1],448,1,2018-05-26 15:18:05
996,2018-05-26,Metzingen,Mentor 4 M,327,2,2018-05-26 15:12:59
997,2018-05-26,Metzingen,Mentor 4 M,471,2,2018-05-26 13:55:31
998,2018-05-26,Metzingen,Anakis3 - XL [LTF 1],372,1,2018-05-26 14:07:50
999,2018-05-26,Metzingen,Anakis3 - XL [LTF 1],253,1,2018-05-26 14:29:28


## Merge wheather and flight data

* join on date
* keep weekends only

In [32]:
df_dhv= df_results.groupby('FlightDate').agg({'Duration':'count','FlightStartTime': lambda x: datetime.strptime(x.max(), "%Y-%m-%d %H:%M:%S") - datetime.strptime(x.min(), "%Y-%m-%d %H:%M:%S") }).reset_index()
df_dhv['Date']=df_dhv.apply(lambda x: datetime.strptime(x['FlightDate'],'%Y-%m-%d'),axis=1)
#print(df_dhv.head())

df_merged = df_tool.merge(df_dhv,on='Date',how='left')
df_merged.rename(columns={'Duration':'flightdays','Date':'predicteddays'}, inplace=True)

dfx =df_merged[(df_merged['Weekday'].isin(['Sat','Sun']))].groupby('Message')[['predicteddays','flightdays']].count().reset_index()


tp = dfx[(dfx['Message'].str.endswith('Std. Flugwetter'))]['flightdays'].sum()
fp = dfx[(dfx['Message'].str.endswith('Std. Flugwetter'))]['predicteddays'].sum() - tp
fn = dfx[~(dfx['Message'].str.endswith('Std. Flugwetter'))]['flightdays'].sum()
print("Flugwetter" + " "*15 + ": precision ", round(tp/(tp+fp),3), " recall ",  round(tp/(tp+fn),3))

tp = dfx[(dfx['Message'].str.endswith('Std. Flugwetter')) | (dfx['Message'].str.endswith('grenzwertig')) ]['flightdays'].sum()
fp = dfx[(dfx['Message'].str.endswith('Std. Flugwetter')) | (dfx['Message'].str.endswith('grenzwertig')) ]['predicteddays'].sum() - tp
fn = dfx[(dfx['Message']=='war wohl kein Flugwetter')]['flightdays'].sum()
print("Flugwetter + grenzwertig : precision ",  round(tp/(tp+fp),3), " recall ",  round(tp/(tp+fn),3))


print(dfx.to_markdown())

Flugwetter               : precision  0.5  recall  0.357
Flugwetter + grenzwertig : precision  0.286  recall  1.0
|    | Message                                       |   predicteddays |   flightdays |
|---:|:----------------------------------------------|----------------:|-------------:|
|  0 | es gab 4 Std. Flugwetter                      |               1 |            1 |
|  1 | es gab 5 Std. Flugwetter                      |               3 |            2 |
|  2 | es gab 7 Std. Flugwetter                      |               1 |            0 |
|  3 | es gab 9 Std. Flugwetter                      |               1 |            1 |
|  4 | es gab wohl nur 1 Std. Flugwetter             |               4 |            1 |
|  5 | vielleicht 4 Std. Flugwetter, grenzwertig     |              12 |            1 |
|  6 | vielleicht 5 Std. Flugwetter, grenzwertig     |               3 |            0 |
|  7 | vielleicht 6 Std. Flugwetter, grenzwertig     |               6 |            1 |
|  8 |